# **Image Dataset**

## Importação de pacotes e leitura dos dados

In [10]:
import numpy as np
import pandas as pd
import os
import imageio


### Função auxiliar para separação dos conjuntos de dados

In [2]:
def get_features_target_data(df, indexes_path):

    indexes = pd.read_csv(indexes_path)

    filtered_df = df.filter(items = indexes.sample_index, axis=0)

    X = filtered_df.drop(columns= 'label')
    y = filtered_df['label']

    return X, y

def get_train_val_set_data(df):

    X_train, y_train = get_features_target_data(df, '../dataset/Data/train_indexes.csv')
    X_val, y_val = get_features_target_data(df, '../dataset/Data/val_indexes.csv')
    X_test, y_test = get_features_target_data(df, '../dataset/Data/test_indexes.csv')

    return X_train, y_train, X_val, y_val, X_test, y_test

### Leitura dos dados e separação em treino, validação e test

In [16]:
from PIL import Image

folder_path = '../dataset/Data/images_original/'
genre_folders = os.listdir(folder_path)

for genre_folder in genre_folders:
  
  files = os.listdir(folder_path + genre_folder)
  print(genre_folder, len(files))
  
  for image in files:
    
    png = Image.open(folder_path + genre_folder+ "/" +image)
    png.load() # required for png.split()

    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel

    background.save('../dataset/Data/images_rgb/' + image, 'JPEG')

blues 100
classical 100
country 100
disco 100
hiphop 100
jazz 99
metal 100
pop 100
reggae 100
rock 100


In [17]:
folder_path = '../dataset/Data/images_rgb/'
genre_folders = os.listdir(folder_path)

In [18]:


file_values = []
file_name = []

for genre_folder in genre_folders:
  
  files = os.listdir(folder_path + genre_folder)
  print(genre_folder, len(files))
  
  for image in files:
    im = imageio.imread(folder_path + genre_folder+ "/" +image)
    file_values.append(im)
    file_name.append(image)

NotADirectoryError: [WinError 267] The directory name is invalid: '../dataset/Data/images_rgb/blues00000.png'

In [13]:
df_dict = {
    'file_name': file_name,
    'file_values': file_values
}

df = pd.DataFrame(df_dict)

In [14]:
df.head()

,file_name,file_values
0,blues00000.png,"[[[255, 255, 255, 255], [255, 255, 255, 255], ..."
1,blues00001.png,"[[[255, 255, 255, 255], [255, 255, 255, 255], ..."
2,blues00002.png,"[[[255, 255, 255, 255], [255, 255, 255, 255], ..."
3,blues00003.png,"[[[255, 255, 255, 255], [255, 255, 255, 255], ..."
4,blues00004.png,"[[[255, 255, 255, 255], [255, 255, 255, 255], ..."


In [3]:
dataset = pd.read_csv('../dataset/Data/features_30_sec.csv')

In [4]:
X_train, y_train, X_val, y_val, X_test, y_test = get_train_val_set_data(dataset)